In [1]:
import findspark
import pyspark
import random
import pandas as pd

In [10]:
from pyspark.sql.functions import *

In [23]:
from datetime import datetime

In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext('local','example')  # if using locally
sqlContext = SQLContext(sc)

In [4]:
df_train_active = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').option('mode','FAILFAST').option("multiLine",'true').option('escape','"').load('gs://dataproc-e3bd1f7b-2e29-4da6-a5c4-077c164fd32a-us-central1/avito/test/train_active.csv')

In [5]:
type(df_train_active)

pyspark.sql.dataframe.DataFrame

In [5]:
df_train_active.show(3)

+------------+------------+--------------------+---------------+--------------------+--------------------+-------+------------------+---------+--------------------+--------------------+--------+---------------+-------------------+---------+
|     item_id|     user_id|              region|           city|parent_category_name|       category_name|param_1|           param_2|  param_3|               title|         description|   price|item_seq_number|    activation_date|user_type|
+------------+------------+--------------------+---------------+--------------------+--------------------+-------+------------------+---------+--------------------+--------------------+--------+---------------+-------------------+---------+
|70d6342079ef|11d4c04d8c51|Нижегородская обл...|Нижний Новгород|       Хобби и отдых|     Книги и журналы|  Книги|              null|     null|Детская знциклопедия|детская энциклопе...|   200.0|           20.0|2017-03-15 00:00:00|  Private|
|97cf505710e1|d7068c60b2c6| Воронежс

In [6]:
df_train_active.printSchema()

root
 |-- item_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- parent_category_name: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- param_1: string (nullable = true)
 |-- param_2: string (nullable = true)
 |-- param_3: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- price: double (nullable = true)
 |-- item_seq_number: double (nullable = true)
 |-- activation_date: timestamp (nullable = true)
 |-- user_type: string (nullable = true)



In [7]:
df_train_active.count()

14129821

In [8]:
len(df_train_active.columns), df_train_active.columns

(15,
 ['item_id',
  'user_id',
  'region',
  'city',
  'parent_category_name',
  'category_name',
  'param_1',
  'param_2',
  'param_3',
  'title',
  'description',
  'price',
  'item_seq_number',
  'activation_date',
  'user_type'])

In [9]:
df_train_active.describe().show()

+-------+------------+------------+-------------------+----------+--------------------+----------------+------------+------------------+------------------+--------------------+--------------------+-------------------+-----------------+---------+
|summary|     item_id|     user_id|             region|      city|parent_category_name|   category_name|     param_1|           param_2|           param_3|               title|         description|              price|  item_seq_number|user_type|
+-------+------------+------------+-------------------+----------+--------------------+----------------+------------+------------------+------------------+--------------------+--------------------+-------------------+-----------------+---------+
|  count|    14129821|    14129821|           14064285|  14064285|            14064285|        14064284|    13513497|           8086250|           5951345|            14063264|            13534224|           13777564|         14005498| 14005498|
|   mean|    Inf

In [11]:
df_train_active_2 = df_train_active.filter(df_train_active.category_name.isNotNull())

In [12]:
df_train_active_2 = df_train_active_2.filter(df_train_active_2.parent_category_name.isNotNull())

In [13]:
df_train_active_2 = df_train_active_2.filter(df_train_active_2.item_id.isNotNull())

In [14]:
df_train_active_2 = df_train_active_2.filter(df_train_active_2.user_id.isNotNull())

In [15]:
df_train_active_2 = df_train_active_2.filter(df_train_active_2.region.isNotNull())

In [16]:
df_train_active_2 = df_train_active_2.filter(df_train_active_2.activation_date.isNotNull())

In [17]:
df_train_active_2 = df_train_active_2.filter(df_train_active_2.user_type.isNotNull())

In [18]:
df_train_active_2 = df_train_active_2.filter(df_train_active_2.item_seq_number.isNotNull())

In [19]:
df_train_active_2 = df_train_active_2.filter(df_train_active_2.title.isNotNull())

In [20]:
df_train_active_2 = df_train_active_2.fillna( { 'price':0 } )

In [21]:
df_train_active_2.take(2)

[Row(item_id='70d6342079ef', user_id='11d4c04d8c51', region='Нижегородская область', city='Нижний Новгород', parent_category_name='Хобби и отдых', category_name='Книги и журналы', param_1='Книги', param_2=None, param_3=None, title='Детская знциклопедия', description='детская энциклопедия,10 томов, в книге 630 страниц ,книги в твердом переплете (22х27) Выпуск-19651968гг Один том-200р', price=200.0, item_seq_number=20.0, activation_date=datetime.datetime(2017, 3, 15, 0, 0), user_type='Private'),
 Row(item_id='97cf505710e1', user_id='d7068c60b2c6', region='Воронежская область', city='Воронеж', parent_category_name='Недвижимость', category_name='Гаражи и машиноместа', param_1='Продам', param_2='Гараж', param_3='Кирпичный', title='Гараж, > 30 м²', description='Продается ГАРАЖ в ПГСК Алмаз/\nУдобный въезд, рядом с пропускным пунктом со стороны ЖК Острова./\nПлощадь 41.   В 2х уровнях./\nСухой не промерзающий подвал. Металлические ворота, есть электричество./\nТерритория охраняется. Возможен 

In [27]:
df_train_active_2.describe().show()

+-------+------------+------------+-------------------+--------+--------------------+----------------+------------+------------------+------------------+--------------------+--------------------+-------------------+-----------------+---------+
|summary|     item_id|     user_id|             region|    city|parent_category_name|   category_name|     param_1|           param_2|           param_3|               title|         description|              price|  item_seq_number|user_type|
+-------+------------+------------+-------------------+--------+--------------------+----------------+------------+------------------+------------------+--------------------+--------------------+-------------------+-----------------+---------+
|  count|    14005498|    14005498|           14005498|14005498|            14005498|        14005498|    13454711|           8086250|           5951345|            14004478|            13475438|           13777564|         14005498| 14005498|
|   mean|    Infinity|  

In [22]:
df_train_active_2.count()

14004478

In [24]:
df_train_active_3 = df_train_active_2.withColumn("activation_date_year",year(df_train_active_2.activation_date))

In [25]:
df_train_active_3 = df_train_active_3.withColumn("activation_date_month",month(df_train_active_3.activation_date))

In [26]:
df_train_active_3 = df_train_active_3.withColumn("activation_date_date",dayofmonth(df_train_active_3.activation_date).alias('dt_day'))

In [27]:
df_train_active_3 = df_train_active_3.withColumn("activation_date_week_no",weekofyear(df_train_active_3.activation_date))

In [28]:
funcWeekDay =  udf(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%w'))

In [29]:
df_train_active_3 = df_train_active_3.withColumn('shortdate',col('activation_date').substr(1, 10))\
     .withColumn('activation_date_week_day', funcWeekDay(col('shortdate')))\
     .drop('shortdate')

In [30]:
df_train_active_3.take(1)

[Row(item_id='70d6342079ef', user_id='11d4c04d8c51', region='Нижегородская область', city='Нижний Новгород', parent_category_name='Хобби и отдых', category_name='Книги и журналы', param_1='Книги', param_2=None, param_3=None, title='Детская знциклопедия', description='детская энциклопедия,10 томов, в книге 630 страниц ,книги в твердом переплете (22х27) Выпуск-19651968гг Один том-200р', price=200.0, item_seq_number=20.0, activation_date=datetime.datetime(2017, 3, 15, 0, 0), user_type='Private', activation_date_year=2017, activation_date_month=3, activation_date_date=15, activation_date_week_no=11, activation_date_week_day='3')]

In [34]:
parent_category_name_map = {"Личные вещи" : "Personal belongings",
                            "Для дома и дачи" : "For the home and garden",
                            "Бытовая электроника" : "Consumer electronics",
                            "Недвижимость" : "Real estate",
                            "Хобби и отдых" : "Hobbies & leisure",
                            "Транспорт" : "Transport",
                            "Услуги" : "Services",
                            "Животные" : "Animals",
                            "Для бизнеса" : "For business"}

region_map = {"Свердловская область" : "Sverdlovsk oblast",
            "Самарская область" : "Samara oblast",
            "Ростовская область" : "Rostov oblast",
            "Татарстан" : "Tatarstan",
            "Волгоградская область" : "Volgograd oblast",
            "Нижегородская область" : "Nizhny Novgorod oblast",
            "Пермский край" : "Perm Krai",
            "Оренбургская область" : "Orenburg oblast",
            "Ханты-Мансийский АО" : "Khanty-Mansi Autonomous Okrug",
            "Тюменская область" : "Tyumen oblast",
            "Башкортостан" : "Bashkortostan",
            "Краснодарский край" : "Krasnodar Krai",
            "Новосибирская область" : "Novosibirsk oblast",
            "Омская область" : "Omsk oblast",
            "Белгородская область" : "Belgorod oblast",
            "Челябинская область" : "Chelyabinsk oblast",
            "Воронежская область" : "Voronezh oblast",
            "Кемеровская область" : "Kemerovo oblast",
            "Саратовская область" : "Saratov oblast",
            "Владимирская область" : "Vladimir oblast",
            "Калининградская область" : "Kaliningrad oblast",
            "Красноярский край" : "Krasnoyarsk Krai",
            "Ярославская область" : "Yaroslavl oblast",
            "Удмуртия" : "Udmurtia",
            "Алтайский край" : "Altai Krai",
            "Иркутская область" : "Irkutsk oblast",
            "Ставропольский край" : "Stavropol Krai",
            "Тульская область" : "Tula oblast"}


category_map = {"Одежда, обувь, аксессуары":"Clothing, shoes, accessories",
"Детская одежда и обувь":"Children's clothing and shoes",
"Товары для детей и игрушки":"Children's products and toys",
"Квартиры":"Apartments",
"Телефоны":"Phones",
"Мебель и интерьер":"Furniture and interior",
"Предложение услуг":"Offer services",
"Автомобили":"Cars",
"Ремонт и строительство":"Repair and construction",
"Бытовая техника":"Appliances",
"Товары для компьютера":"Products for computer",
"Дома, дачи, коттеджи":"Houses, villas, cottages",
"Красота и здоровье":"Health and beauty",
"Аудио и видео":"Audio and video",
"Спорт и отдых":"Sports and recreation",
"Коллекционирование":"Collecting",
"Оборудование для бизнеса":"Equipment for business",
"Земельные участки":"Land",
"Часы и украшения":"Watches and jewelry",
"Книги и журналы":"Books and magazines",
"Собаки":"Dogs",
"Игры, приставки и программы":"Games, consoles and software",
"Другие животные":"Other animals",
"Велосипеды":"Bikes",
"Ноутбуки":"Laptops",
"Кошки":"Cats",
"Грузовики и спецтехника":"Trucks and buses",
"Посуда и товары для кухни":"Tableware and goods for kitchen",
"Растения":"Plants",
"Планшеты и электронные книги":"Tablets and e-books",
"Товары для животных":"Pet products",
"Комнаты":"Room",
"Фототехника":"Photo",
"Коммерческая недвижимость":"Commercial property",
"Гаражи и машиноместа":"Garages and Parking spaces",
"Музыкальные инструменты":"Musical instruments",
"Оргтехника и расходники":"Office equipment and consumables",
"Птицы":"Birds",
"Продукты питания":"Food",
"Мотоциклы и мототехника":"Motorcycles and bikes",
"Настольные компьютеры":"Desktop computers",
"Аквариум":"Aquarium",
"Охота и рыбалка":"Hunting and fishing",
"Билеты и путешествия":"Tickets and travel",
"Водный транспорт":"Water transport",
"Готовый бизнес":"Ready business",
"Недвижимость за рубежом":"Property abroad"}

In [35]:
funcParentCatMap =  udf(lambda x: parent_category_name_map[x])
funcCatNameMap =  udf(lambda x: category_map[x])
funcRegionMap =  udf(lambda x: region_map[x])

In [36]:
df_train_active_3 = df_train_active_3.withColumn('parent_category_name_en', funcParentCatMap(col('parent_category_name')))
df_train_active_3 = df_train_active_3.withColumn('category_name_en', funcCatNameMap(col('category_name')))
df_train_active_3 = df_train_active_3.withColumn('region_en', funcRegionMap(col('region')))

In [37]:
df_train_active_3.show(3)

+------------+------------+--------------------+---------------+--------------------+--------------------+-------+------------------+---------+--------------------+--------------------+--------+---------------+-------------------+---------+--------------------+---------------------+--------------------+-----------------------+------------------------+-----------------------+--------------------+--------------------+
|     item_id|     user_id|              region|           city|parent_category_name|       category_name|param_1|           param_2|  param_3|               title|         description|   price|item_seq_number|    activation_date|user_type|activation_date_year|activation_date_month|activation_date_date|activation_date_week_no|activation_date_week_day|parent_category_name_en|    category_name_en|           region_en|
+------------+------------+--------------------+---------------+--------------------+--------------------+-------+------------------+---------+-----------------

In [38]:
df_train_active_3.select('parent_category_name_en').distinct().count()

9

In [39]:
df_train_active_3.select('category_name_en').distinct().count()

47

In [40]:
df_train_active_3.select('region_en').distinct().count()

28

In [41]:
df_train_active_3.select('parent_category_name_en').distinct().show()

+-----------------------+
|parent_category_name_en|
+-----------------------+
|               Services|
|            Real estate|
|      Hobbies & leisure|
|           For business|
|   Consumer electronics|
|    Personal belongings|
|                Animals|
|              Transport|
|   For the home and ...|
+-----------------------+



In [42]:
df_train_active_3.select('category_name_en').distinct().show()

+--------------------+
|    category_name_en|
+--------------------+
|Repair and constr...|
|          Apartments|
|            Aquarium|
|                Food|
|Sports and recrea...|
| Watches and jewelry|
|       Other animals|
|  Tickets and travel|
|      Offer services|
|      Ready business|
|               Bikes|
|               Photo|
|        Pet products|
| Books and magazines|
| Musical instruments|
|   Health and beauty|
|Houses, villas, c...|
|Children's clothi...|
|                Land|
|Clothing, shoes, ...|
+--------------------+
only showing top 20 rows



In [43]:
df_train_active_3.select('region_en').distinct().show()

+--------------------+
|           region_en|
+--------------------+
|       Samara oblast|
|      Krasnodar Krai|
|           Tatarstan|
|Khanty-Mansi Auto...|
|     Belgorod oblast|
|         Omsk oblast|
|Nizhny Novgorod o...|
|       Tyumen oblast|
|    Yaroslavl oblast|
|    Krasnoyarsk Krai|
|       Rostov oblast|
|     Voronezh oblast|
|         Tula oblast|
|     Kemerovo oblast|
|  Chelyabinsk oblast|
|          Altai Krai|
|  Kaliningrad oblast|
|      Stavropol Krai|
|       Bashkortostan|
|     Vladimir oblast|
+--------------------+
only showing top 20 rows



In [45]:
df_train_active_3.write.parquet("gs://dataproc-e3bd1f7b-2e29-4da6-a5c4-077c164fd32a-us-central1/avito/neat_data/train_active.parquet")

In [46]:
df_train_active_4 = sqlContext.read.parquet("gs://dataproc-e3bd1f7b-2e29-4da6-a5c4-077c164fd32a-us-central1/avito/neat_data/train_active.parquet")

In [47]:
df_train_active_4.show(3)

+------------+------------+--------------------+---------------+--------------------+--------------------+-------+------------------+---------+--------------------+--------------------+--------+---------------+-------------------+---------+--------------------+---------------------+--------------------+-----------------------+------------------------+-----------------------+--------------------+--------------------+
|     item_id|     user_id|              region|           city|parent_category_name|       category_name|param_1|           param_2|  param_3|               title|         description|   price|item_seq_number|    activation_date|user_type|activation_date_year|activation_date_month|activation_date_date|activation_date_week_no|activation_date_week_day|parent_category_name_en|    category_name_en|           region_en|
+------------+------------+--------------------+---------------+--------------------+--------------------+-------+------------------+---------+-----------------

In [48]:
df_train_active_4.count()

14004478